In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from cr.sparse.io import print_dataframe_as_list_table

In [4]:
K = 20
M = 200
N = 1000
bits = 64

In [5]:
filename = f'record_benchmarks_{64}bit_M_{M}_N_{N}_K_{K}.csv'
filename

'record_benchmarks_64bit_M_200_N_1000_K_20.csv'

In [6]:
df = pd.read_csv(filename)

In [7]:
df.head()

,m,n,k,method,jit,success,iterations,recovery_snr,measurement_snr,k_recovered,average,best,worst,stdev,loops,repeat
0,200,1000,20,OMP,False,True,20,315.292850,313.743883,20,112.692723,107.871920,116.872663,3.453383,10,7
1,200,1000,20,OMP,True,True,20,309.328057,308.885738,20,2.427774,2.393887,2.443323,0.018012,100,7
2,200,1000,20,SP,False,True,4,296.387025,295.918869,20,1324.785409,1268.595889,1355.435483,30.629732,1,7
3,200,1000,20,SP,True,True,4,296.387025,295.918869,20,4.490560,4.322779,4.637684,0.108696,100,7
4,200,1000,20,CoSaMP,False,True,5,300.080706,299.123450,20,293.501159,275.459946,319.790643,13.508331,1,7


In [8]:
# Replace True/False with on off in jit column
df['jit'] = np.where(df['jit'], 'jit_on', 'jit_off')

In [9]:
# Identity the names of methods used
methods = list(df['method'])

In [10]:
# Remove duplicate names while retaining order
unique_methods = [i for n, i in enumerate(methods) if i not in methods[:n]]
unique_methods

['OMP', 'SP', 'CoSaMP', 'IHT', 'NIHT', 'HTP', 'NHTP']

In [11]:
# Pivot on the jit column
df3 = df.pivot(index="method", columns='jit', values=['iterations', 'average'])

In [12]:
# Retain the original order of method names
df3 = df3.loc[unique_methods]

In [13]:
df3.columns = df3.columns.to_flat_index()

In [14]:
df4 = df3.drop([('iterations', 'jit_on')], axis = 1)

In [15]:
df5 = df4.rename(columns={('average', 'jit_off'): 'jit_off', ('average', 'jit_on'): 'jit_on', ('iterations', 'jit_off'): 'iterations'})

In [16]:
df5

,iterations,jit_off,jit_on
method,,,
OMP,20.0,112.692723,2.427774
SP,4.0,1324.785409,4.490560
CoSaMP,5.0,293.501159,9.816549
IHT,77.0,209.224982,48.805604
NIHT,19.0,196.657989,7.226292
HTP,6.0,1218.616871,18.957674
NHTP,5.0,1238.366780,2.791099


In [17]:
# Measure speed up
df5['speedup'] = df5['jit_off'] / df5['jit_on']

In [18]:
df5.insert(0,'M', M)
df5.insert(1,'N', N)
df5.insert(2,'K', K)
df5

,M,N,K,iterations,jit_off,jit_on,speedup
method,,,,,,,
OMP,200,1000,20,20.0,112.692723,2.427774,46.418130
SP,200,1000,20,4.0,1324.785409,4.490560,295.015635
CoSaMP,200,1000,20,5.0,293.501159,9.816549,29.898608
IHT,200,1000,20,77.0,209.224982,48.805604,4.286905
NIHT,200,1000,20,19.0,196.657989,7.226292,27.214234
HTP,200,1000,20,6.0,1218.616871,18.957674,64.280928
NHTP,200,1000,20,5.0,1238.366780,2.791099,443.684277


In [19]:
df5['iterations'] = df5['iterations'].astype(int)

In [20]:
df5

,M,N,K,iterations,jit_off,jit_on,speedup
method,,,,,,,
OMP,200,1000,20,20,112.692723,2.427774,46.418130
SP,200,1000,20,4,1324.785409,4.490560,295.015635
CoSaMP,200,1000,20,5,293.501159,9.816549,29.898608
IHT,200,1000,20,77,209.224982,48.805604,4.286905
NIHT,200,1000,20,19,196.657989,7.226292,27.214234
HTP,200,1000,20,6,1218.616871,18.957674,64.280928
NHTP,200,1000,20,5,1238.366780,2.791099,443.684277


In [21]:
# Convert the table for Sphinxdoc format
print_dataframe_as_list_table(df5, f'Average time (msec) and speedups due to JIT compilation')

.. list-table:: Average time (msec) and speedups due to JIT compilation
    :header-rows: 1

    * - method
      - M
      - N
      - K
      - iterations
      - jit_off
      - jit_on
      - speedup
    * - OMP
      - 200
      - 1000
      - 20
      - 20
      - 112.69
      - 2.43
      - 46.42
    * - SP
      - 200
      - 1000
      - 20
      - 4
      - 1324.79
      - 4.49
      - 295.02
    * - CoSaMP
      - 200
      - 1000
      - 20
      - 5
      - 293.50
      - 9.82
      - 29.90
    * - IHT
      - 200
      - 1000
      - 20
      - 77
      - 209.22
      - 48.81
      - 4.29
    * - NIHT
      - 200
      - 1000
      - 20
      - 19
      - 196.66
      - 7.23
      - 27.21
    * - HTP
      - 200
      - 1000
      - 20
      - 6
      - 1218.62
      - 18.96
      - 64.28
    * - NHTP
      - 200
      - 1000
      - 20
      - 5
      - 1238.37
      - 2.79
      - 443.68
